In [1]:
import os
path_peer = 'LibriSpeech/'
is_peer = os.path.exists(path_peer)

if is_peer:
    os.chdir(path_peer)

print('Directory is set to', os.getcwd())

Directory is set to /home/eva/eva_project_dir/VAD/LibriSpeech


In [2]:
import pandas as pd
import numpy as np

import sys

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from function import * 

from IPython.display import Audio

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
import glob
import re
from pydub import AudioSegment
import soundfile as sf
import h5py
import webrtcvad


In [4]:
SAMPLE_RATE = 16000
SAMPLE_CHANNELS = 1
SAMPLE_WIDTH = 2

# Name of folder to save the data files in.
DATA_FOLDER = 'data'

# Min/max length for slicing the voice files.
SLICE_MIN_MS = 1000
SLICE_MAX_MS = 5000

# Размер фрейма
FRAME_SIZE_MS = 10

# Convert slice ms to frame size.
SLICE_MIN = int(SLICE_MIN_MS / FRAME_SIZE_MS)
SLICE_MAX = int(SLICE_MAX_MS / FRAME_SIZE_MS)

# Calculate frame size in data points.
FRAME_SIZE = int(SAMPLE_RATE * (FRAME_SIZE_MS / 1000.0))

In [5]:
data = h5py.File('data_noise.hdf5', 'a')

In [6]:
files = glob.glob('../noize_raw/***/**/*.wav', recursive=True)

In [7]:
files = [f for f in files]

#Выделяем пространство для датасета с путями файлов
dt = h5py.special_dtype(vlen=str)
data.create_dataset('files', (len(files),), dtype=dt)

#Записываем пути файлов 
for i, f in enumerate(files):
    data['files'][i] = f

In [8]:
def get_track_count():
    return len(data['files'])

In [9]:
#Конвентируем аудиофайлы в нужный формат 
#Сохраняем каждый файл в отдельный датасет 'audio_raw'

progress = 1
# Создаем пространство для адуио в формате int16! Такой формат необходим для WebRTCVAD
if 'audio_raw' not in data:
    dt = h5py.special_dtype(vlen=np.dtype(np.int16))
    data.create_dataset('audio_raw', (get_track_count(),), dtype=dt)

for i, file in enumerate(data['files']): 
    print('Processing {0} of {1}'.format(progress, get_track_count()), end='\r', flush=True)
    progress += 1
    # Проверяем сконвертировано ли уже аудио
    if len(data['audio_raw'][i]) > 0:
        continue

    track = (AudioSegment.from_file((file).decode("utf-8"))
             .set_frame_rate(SAMPLE_RATE)
             .set_sample_width(SAMPLE_WIDTH)
             .set_channels(SAMPLE_CHANNELS))

    data['audio_raw'][i] = np.array(track.get_array_of_samples(), dtype=np.int16)

In [10]:
#проверяем, что все записалось верно и с форматом int16
data['audio_raw'][0]

array([  0,  -8, -17, ...,   0,   0,   0], dtype=int16)

In [11]:
%%time
frame_count = 0
progress = 1

# Вычисляем кол-во итоговых фреймов, которые получаются из всех записей, смердженных вместе
for raw in data['audio_raw']:
    frame_count += int((len(raw) + (FRAME_SIZE - (len(raw) % FRAME_SIZE))) / FRAME_SIZE)
    print('Counting frames ({0} of {1})'.format(progress, get_track_count()), end='\r', flush=True)
    progress += 1

# Создаем датасет необходимого размера для хранения фреймов: кол-во фреймов, размер одного фрейма
dt = np.dtype(np.int16)
data.create_dataset('frames', (frame_count, FRAME_SIZE), dtype=dt)

progress = 0

# Buffer to speed up merging as HDF5 is not fast with lots of indexing.
buffer = np.array([])
buffer_limit = FRAME_SIZE * 4096

# Merge frames.
for raw in data['audio_raw']:

    # Добавляем паддинг, чтобы каждая запись разбивалась на фреймы без остатка
    raw = np.concatenate((raw, np.zeros(FRAME_SIZE - (len(raw) % FRAME_SIZE))))

    # Add to buffer.
    buffer = np.concatenate((buffer, raw))

    # If buffer is not filled up and we are not done, keep filling the buffer up.
    if len(buffer) < buffer_limit and progress + (len(buffer) / FRAME_SIZE) < frame_count:
        continue

    # Получаем фреймы
    frames = np.array(np.split(buffer, len(buffer) / FRAME_SIZE))
    buffer = np.array([])

    # Добавляем фреймы 
    data['frames'][progress : progress + len(frames)] = frames

    progress += len(frames)
    print('Merging frames ({0} of {1})'.format(progress, frame_count), end='\r', flush=True)

CPU times: user 15.8 s, sys: 10.3 s, total: 26.1 s
Wall time: 45.2 s


In [1]:
%%time
vad = webrtcvad.Vad(0)

frame_count = len(data['frames'])
progress = 0
batch_size = 65536

# Create data set for labels.
dt = np.dtype(np.uint8)
data.create_dataset('labels_new', (frame_count,), dtype=dt)

# Label all the frames.
for pos in range(0, frame_count, batch_size):

    frames = data['frames'][pos : pos + batch_size]
    labels = [1 if vad.is_speech(f.tobytes(), sample_rate=SAMPLE_RATE) else 0 for f in frames]
    data['labels_new'][pos : pos + batch_size] = np.array(labels)

    progress += len(labels)
    print('Labelling frames ({0} of {1})'.format(progress, frame_count), end='\r', flush=True)



NameError: name 'webrtcvad' is not defined

In [13]:
import h5py_cache

In [14]:
data = h5py_cache.File('data_noise.hdf5', 'a', chunk_cache_mem_size=1024**3)